# Connect Playlists and Locations

In [3]:
import pandas as pd
import time
import json
import googlemaps
import homebrew as hb
from sklearn.neighbors import NearestNeighbors

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_json('artists_shows.json', lines=True)

In [4]:
print(df.shape)
df.head()

(134943, 3)


,artist,date,loc
0,2 Chainz,2019-06-30 05:30:00,"Las Vegas, NV, US"
1,2 Chainz,2019-07-12 11:00:00,"London, UK"
2,2 Chainz,2019-08-10 00:00:00,"Anaheim, CA, US"
3,2 Chainz,2019-08-24 22:00:00,"Atlanta, GA, US"
4,2 Chainz,2019-05-27 05:30:00,"Las Vegas, NV, US"


In [5]:
df.date.describe()

count                  134943
unique                  37136
top       2019-07-19 00:00:00
freq                      191
first     1973-09-29 00:00:00
last      2020-12-16 18:30:00
Name: date, dtype: object

In [6]:
df.columns = ['artist', 'date', 'location']    # rename loc -> location
domestics = df[df['location'].str.endswith('US')]    # drop any foreign gigs
recent_domestics = domestics[domestics['date'] >= '2015']    #  drop any gigs older than 5 yrs
locations = recent_domestics.copy()

In [7]:
unique_locations = locations['location'].unique()
f'There are {unique_locations.shape[0]} unique locations'

'There are 2023 unique locations'

Get lat longs from googlemaps

In [8]:
# unique_loc_w_latlong = []

# for i, location in enumerate(unique_locations):
#     row = {}
#     lat, lng = hb.get_latlong(location)
#     row['location'] = location
#     row['lat'] = lat
#     row['lng'] = lng
#     unique_loc_w_latlong.append(row)
#     if i % 50 == 0:
#         time.sleep(2)
#     if i % 100 == 0:
#         print(i)                # Slowish 

In [9]:
latlongs_df = pd.DataFrame(unique_loc_w_latlong)
latlongs_df.head()

NameError: name 'unique_loc_w_latlong' is not defined

In [61]:
shows_w_latlong = pd.merge(locations, latlongs_df, on='location', how='left')
print(locations.shape)
shows_w_latlong.shape

(75699, 3)


(75699, 5)

In [62]:
shows_w_latlong.to_json('shows_w_latlong.json')

### Look for Similar Artists

In [4]:
shows_w_latlong = pd.read_json('shows_w_latlong.json')

In [5]:
artists_playlists = pd.read_json('trimmed_artists_w_flags.json', lines=True)

In [6]:
artists_playlists['artist'] = artists_playlists['artist'].map(str.lower)
shows_w_latlong['artist'] = shows_w_latlong['artist'].map(str.lower)

In [7]:
similar_artist_model = hb.SimilarArtistModel(artists_playlists)

In [8]:
artists_playlists[artists_playlists['artist']=='van morrison']
# shows_w_latlong[shows_w_latlong['artist']=='van morrison']

,artist,playlist_08CJ6fWrTA1vMfwVwAxWUl,playlist_0Gy1TwCxPFTMCiwxGLVkc1,playlist_0T22esVKW7cTZEN7XzXV14,playlist_0YMVY3uhQNd7WUf0hSByrX,playlist_0ZWYUjUMhaq0Rammqq3qu1,playlist_0bx5Dgu5yOEhST49LP67su,playlist_0fmLn46c92nP4dePQSZlsz,playlist_0k7YwkrfJ2BsRMFBP5W5xK,playlist_0yfTPCk6Cnhm9wtoEaXXlo,...,playlist_73gXbLcjix7p1pPsQClusE,playlist_7A2YimOfIrmAWkCeSIY8Rq,playlist_7EnyqmmhFgfhicejZiVV13,playlist_7EpJpnU6KrSc430F8B5tM7,playlist_7FDyC1JdV3jkGzAE9BONKt,playlist_7HQu1GUDVSx64GdCpaB88I,playlist_7JXJKP3xMvIMbEBBGApTPQ,playlist_7b9DiMN2kttG8lYQogfQBw,playlist_7fzFQUIVM4TumzRWo5UFP2,playlist_7kdOsNnHtzwncTBnI3J17w
1601,van morrison,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
main_artist = input("Enter Artist:").lower()
num_artists = 2
if main_artist in shows_w_latlong['artist'].values:
    similar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists)

    for artist_tup in similar_artists:
        artist, cosine_sim = artist_tup
        print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
        new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                             main_artist=main_artist,
                                             similar_artist=artist,
                                             max_new_cities=10)
        for spot in new_spots:
            print('\t', spot)
else:
    print(f'{main_artist.title()}: No Recent Shows OR Not In 20 Of Top Playlists')

Enter Artist: the black keys


Similar Band: THE STRUTS - - Cosine Similarity: 0.345
	 {'new': 'Missoula, MT, US', 'played': 'Tacoma, WA, US', 'distance': 398.0}
	 {'new': 'Salt Lake City, UT, US', 'played': 'Denver, CO, US', 'distance': 371.0}
	 {'new': 'New Orleans, LA, US', 'played': 'Houston, TX, US', 'distance': 318.0}
	 {'new': 'Mobile, AL, US', 'played': 'Atlanta, GA, US', 'distance': 300.0}
	 {'new': 'Destin, FL, US', 'played': 'Atlanta, GA, US', 'distance': 263.0}
	 {'new': 'Fort Smith, AR, US', 'played': 'Fort Worth, TX, US', 'distance': 247.0}
	 {'new': 'Las Vegas, NV, US', 'played': 'Inglewood, CA, US', 'distance': 237.0}
	 {'new': 'Sioux City, IA, US', 'played': 'Minneapolis, MN, US', 'distance': 232.0}
	 {'new': 'Charleston, SC, US', 'played': 'Raleigh, NC, US', 'distance': 220.0}
	 {'new': 'Tulsa, OK, US', 'played': 'Kansas City, MO, US', 'distance': 218.0}
Similar Band: DES ROCS - - Cosine Similarity: 0.402
	 {'new': 'New Orleans, LA, US', 'played': 'Houston, TX, US', 'distance': 318.0}
	 {'new': 'So

In [84]:
main_artist = input("Greg added this").lower()
num_artists = 5
if main_artist in shows_w_latlong['artist'].values:
    similar_artists = similar_artist_model.find_artists_sim_to(main_artist, n_neighbors=num_artists)
    for artist, cosine_sim in similar_artists:
        print(f'Similar Band: {artist.upper()} - - Cosine Similarity: {cosine_sim}')
        new_spots = hb.pilfer_similar_artist(ref_df=shows_w_latlong,
                                             main_artist=main_artist,
                                             similar_artist=artist,
                                             max_new_cities=3)
        for spot in new_spots:
            print('\t', spot)
else:
    print(f'{main_artist.title()}: No Recent Shows OR Not In 20 Of Top Playlists')

Greg added this arcade fire


Arcade Fire: No Recent Shows OR Not In 20 Of Top Playlists


In [90]:
artist_in_question = 'arcade fire'
similar_artist_model.find_similar_artist_venues(artist_in_question, shows_w_latlong)

'Arcade Fire: No Recent Shows OR Not In 20 Of Top Playlists'

In [68]:
import pickle
!pwd

/Users/patrickfuller/flatiron/capstone/solve_music_probs


In [94]:
with open('sim_artist_model.pkl', 'wb') as f:
    pickle.dump(similar_artist_model, f)